### Non-Linear Models
- Kernal-Rdige Regression
  -  ‘linear’, ‘poly’, ‘rbf’
- SVR
  - 'linear', 'rbf'
- Ensemble Model
  - AdaBoost Regressor
  - Bagging Regressor
  - Gradient Boosting Regressor
  - Random Forest Regressor
- NN
  - MLP Regressor
- Nearest Neighbors
  - KNeighbors Regressor
  
http://scikit-learn.org/stable/modules/classes.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import minmax_scale
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# models
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
def load_data():
    df = pd.read_csv("../data/autos_removed_target_mv.csv")
    return df

# drop all missing vals
def handling_mv(df):
    df = df.replace("?", np.nan)
    df = df.dropna()
    return df

# preprocessing
def preprocessing(df):
    y = df['normalized-losses']
    X = df.ix[:, 1:]
    
    # divide real and cat vals
    cols_real = list(X.describe().columns)
    cols_cat = [col for col in X.columns if col not in cols_real]
    X_real = X[cols_real]
    X_cat = X[cols_cat]
    
    # scaling real vlas (minmax scale; 0~1)
    X_scaled = minmax_scale(X_real)
    dfX_scale = pd.DataFrame(X_scaled, columns=cols_real)

    # dummies cat vals
    X_dummies = pd.get_dummies(X_cat)
    
    # reindex & merge real and cat DF
    dfX_scale.reset_index(drop=True, inplace=True)
    X_dummies.reset_index(drop=True, inplace=True)
    X_pre = pd.concat([dfX_scale, X_dummies], axis=1)
    
    return X_pre, y

df = load_data()
df = handling_mv(df)
X_pre, y = preprocessing(df)

In [3]:
# grid search cv
def modeling(algo, param_grid, cv=None, scoring="neg_mean_squared_error"):    
    model = GridSearchCV(algo,
                        param_grid,
                        cv=cv,
                        scoring=scoring)
    return model.fit(X_pre, y)

# parameter search graph 2D, 3D
def param_search(model):
    rmse = (-model.cv_results_['mean_test_score'])**(1/2)
    print("best param: ", model.best_params_, "\nmin rmse: ", np.min(rmse))

### KERNEL RIDGE

In [4]:
param_grid = {
    "alpha": np.logspace(-2, 1, 50),
    "kernel": ['linear', 'rbf', 'poly', 'sigmoid']
}

kernel_ridge_model = modeling(KernelRidge(), param_grid, cv=10)

In [5]:
param_search(kernel_ridge_model)

best param:  {'alpha': 0.29470517025518095, 'kernel': 'linear'} 
min rmse:  29.496329546858455


### SVR

In [6]:
param_grid = {
    "kernel": ['rbf', 'linear'],
    "C": np.linspace(0.1, 3, 10),
    "epsilon": np.logspace(-2, 0, 20),
    "gamma": np.linspace(1, 10, 10)
}

svr_model = modeling(SVR(), param_grid, cv=5)

In [7]:
param_search(svr_model)

best param:  {'gamma': 1.0, 'C': 3.0, 'epsilon': 0.29763514416313175, 'kernel': 'linear'} 
min rmse:  34.421741348392324


### AdaBoost

In [8]:
param_grid = {
    "learning_rate": np.linspace(0.01, 3, 20),
    "loss": ['linear', 'square', 'exponential']
}

ada_model = modeling(AdaBoostRegressor(), param_grid, cv=5)

In [9]:
param_search(ada_model)

best param:  {'loss': 'linear', 'learning_rate': 1.583684210526316} 
min rmse:  33.842678816341724


### GBM

In [10]:
param_grid = {
    "loss": ['ls', 'lad', 'huber', 'quantile'],
    "learning_rate": np.linspace(0.01, 3, 20),
    "n_estimators": np.arange(1, 10),
    "max_depth": np.arange(1, 5)
}

gbm = modeling(GradientBoostingRegressor(), param_grid, cv=5)

In [11]:
param_search(gbm)

best param:  {'max_depth': 4, 'loss': 'lad', 'n_estimators': 7, 'learning_rate': 1.1115789473684212} 
min rmse:  33.340163292476134


### Bagging

In [12]:
param_grid = {
    "n_estimators": np.arange(1, 10),
    "max_samples": np.arange(10, 20),
    "max_features": np.arange(10, 20),
}

bagging_model = modeling(BaggingRegressor(), param_grid, cv=5)

In [13]:
param_search(bagging_model)

best param:  {'max_features': 10, 'n_estimators': 4, 'max_samples': 10} 
min rmse:  31.194332887314502


### RF

In [14]:
param_grid = {
    "n_estimators": np.arange(1, 10),
    "max_depth": np.arange(1, 5),
}

rf_model = modeling(RandomForestRegressor(), param_grid, cv=5)

In [15]:
param_search(rf_model)

best param:  {'max_depth': 2, 'n_estimators': 9} 
min rmse:  35.50363389125756


### NN

In [16]:
param_grid = {
    "hidden_layer_sizes": [(100, 10)],
    "learning_rate": ['constant', 'invscaling', 'adaptive'],
    "learning_rate_init": np.linspace(0.005, 2, 20)
}

nn_model = modeling(MLPRegressor(), param_grid, cv=5)

In [17]:
param_search(nn_model)

best param:  {'hidden_layer_sizes': (100, 10), 'learning_rate_init': 0.005, 'learning_rate': 'constant'} 
min rmse:  40.77031707642177


### KN

In [18]:
param_grid = {
    "n_neighbors": np.arange(1, 30),
    "weights": ['uniform', 'distance'],
    "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']
}

kn_model = modeling(KNeighborsRegressor(), param_grid, cv=5)

In [19]:
param_search(kn_model)

best param:  {'algorithm': 'brute', 'n_neighbors': 23, 'weights': 'distance'} 
min rmse:  35.41668773068977
